### Library imports

In [1]:
import numpy as np
import pandas as pd
import requests
from math import floor
from scipy import stats
import pprint
from dotenv import dotenv_values
from econ_utils import chunks, check_divide_by_zero, iex_cloud_api
from scipy.stats import percentileofscore as score
from constants import STATS_COLS, MARKET_COLS

config = dotenv_values()
pp = pprint.PrettyPrinter(indent=2)

In [2]:
sp100_df = pd.read_excel("../data/SP100.xlsx")
sp100_df

,Symbol,Company Name,Sector
0,AAPL,Apple,Information Technology
1,ABBV,AbbVie,Health Care
2,ABT,Abbott,Health Care
3,ACN,Accenture,Information Technology
4,ADBE,Adobe,Information Technology
...,...,...,...
96,VZ,Verizon,Communication Services
97,WBA,Walgreens Boots Alliance,Consumer Staples
98,WFC,Wells Fargo,Financials
99,WMT,Walmart,Consumer Staples


### Method for all SP 500 stocks

In [3]:
symbol_chunks = list(chunks(sp100_df["Symbol"], 100))
symbol_csv_list = []

for i in range(0, len(symbol_chunks)):
  symbol_csv_list.append(",".join(symbol_chunks[i]))

In [4]:
market_responses = []
for symbol_csv in symbol_csv_list:
  response = iex_cloud_api("stock/market/batch", symbols=symbol_csv, types="quote")
  market_responses.append(response)

In [7]:
stocks_df = pd.DataFrame(columns=MARKET_COLS.values())

# #1 TODO find out what kind of errors can be thrown here
for response in market_responses:
  for symbol in response.keys():
    try:
      quote = response[symbol]["quote"].values()
      stocks_df.loc[symbol] = pd.Series(quote, index=MARKET_COLS.values())
    except: # #1
      print(f"Symbol {symbol} failed: {quote}")
stocks_df

,Average Total Volume,Calculation Price,Change,Change %,Close,Close Source,Close Time,Company Name,Currency,Delayed Price,...,PE Ratio,Previous Close,Previous Volume,Primary Exchange,Symbol,Volume,52 Week High,52 Week Low,Year-to-date Change,Is US Market Open
AAPL,80930748,tops,-1.14,-0.00685,None,official,None,Apple Inc,USD,None,...,27.46,166.42,87227768,NASDAQ,AAPL,None,182.71,121.73,-0.068448,True
ABBV,7643404,tops,-2.18,-0.01375,None,official,None,Abbvie Inc,USD,None,...,24.24,158.52,7903316,NEW YORK STOCK EXCHANGE INC.,ABBV,None,174.43,102.35,0.179205,True
ABT,4961556,tops,-3.02,-0.02448,None,official,None,Abbott Laboratories,USD,None,...,30.47,123.37,7321143,NEW YORK STOCK EXCHANGE INC.,ABT,None,141.53,103.78,-0.141301,True
ACN,2196120,tops,-5.45,-0.01691,None,official,None,Accenture plc - Class A,USD,None,...,31.97,322.31,1910213,NEW YORK STOCK EXCHANGE INC.,ACN,None,415.05,273.76,-0.23508,True
ADBE,3126202,tops,-2.02,-0.00484,None,official,None,Adobe Inc,USD,None,...,41.22,417.48,2390149,NASDAQ,ADBE,None,699.54,407.94,-0.268622,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VZ,23576485,tops,-2.91,-0.0529,None,official,None,Verizon Communications Inc,USD,None,...,9.79,55.01,32821141,NEW YORK STOCK EXCHANGE INC.,VZ,None,57.74,48.5,0.031601,True
WBA,8093422,tops,-0.71,-0.01517,None,official,None,Walgreens Boots Alliance Inc,USD,None,...,6.36,46.81,7106024,NASDAQ,WBA,None,54.45,42.9,-0.108699,True
WFC,26528316,tops,-0.21,-0.0044,None,official,None,Wells Fargo & Co.,USD,None,...,9.61,47.78,19027637,NEW YORK STOCK EXCHANGE INC.,WFC,None,60.3,40.95,-0.004096,True
WMT,7244939,tops,-0.47,-0.00294,None,official,None,Walmart Inc,USD,None,...,32.73,159.87,6056100,NEW YORK STOCK EXCHANGE INC.,WMT,None,160.77,131.5,0.106248,True


In [9]:
position_size = 1e6
stocks_df["Shares to buy"] = (position_size * stocks_df[MARKET_COLS["latestPrice"]] / stocks_df[MARKET_COLS["latestPrice"]].sum())
top_20_pe_stocks = stocks_df.sort_values(MARKET_COLS["peRatio"], ascending=False).head(20)
top_20_pe_stocks

,Average Total Volume,Calculation Price,Change,Change %,Close,Close Source,Close Time,Company Name,Currency,Delayed Price,...,Previous Close,Previous Volume,Primary Exchange,Symbol,Volume,52 Week High,52 Week Low,Year-to-date Change,Is US Market Open,Shares to buy
TSLA,22880399,tops,4.46,0.00442,None,official,None,Tesla Inc,USD,None,...,1008.78,35138779,NASDAQ,TSLA,None,1243.49,546.98,-0.041001,True,36591.161554
CRM,5852779,tops,-2.91,-0.01642,None,official,None,Salesforce Inc,USD,None,...,177.23,9119512,NEW YORK STOCK EXCHANGE INC.,CRM,None,311.75,174.23,-0.319021,True,6295.222536
BKNG,326574,tops,10.83,0.00484,None,official,None,Booking Holdings Inc,USD,None,...,2239.32,389840,NASDAQ,BKNG,None,2715.66,1796.45,-0.061811,True,81259.723432
DIS,10925277,tops,-0.765,-0.00629,None,official,None,Walt Disney Co (The),USD,None,...,121.66,18688636,NEW YORK STOCK EXCHANGE INC.,DIS,None,189.22,119.7,-0.220829,True,4365.88417
TMUS,4960686,tops,-2.49,-0.01885,None,official,None,T-Mobile US Inc,USD,None,...,132.1,3580745,NASDAQ,TMUS,None,150.2,101.51,0.120139,True,4680.609183
KHC,5819140,tops,-0.075,-0.0017,None,official,None,Kraft Heinz Co,USD,None,...,44,9219449,NASDAQ,KHC,None,44.48,32.44,0.236903,True,1586.264628
NVDA,54915672,tops,-2.51,-0.01244,None,official,None,NVIDIA Corp,USD,None,...,201.83,65620944,NASDAQ,NVDA,None,346.37,134.5,-0.326083,True,7198.048163
COST,2374253,tops,-8.55,-0.01445,None,official,None,Costco Wholesale Corp,USD,None,...,591.74,2027947,NASDAQ,COST,None,612.27,362.69,0.029476,True,21060.755109
AMT,1823217,tops,-2.18,-0.00817,None,official,None,American Tower Corp.,USD,None,...,266.75,1712477,NEW YORK STOCK EXCHANGE INC.,AMT,None,299.22,218.83,-0.091329,True,9554.423051
LLY,2692400,tops,-5.18,-0.01789,None,official,None,Lilly(Eli) & Co,USD,None,...,289.61,2716751,NEW YORK STOCK EXCHANGE INC.,LLY,None,314,175.91,0.034963,True,10271.62773


In [10]:
stats_df = pd.DataFrame(columns=STATS_COLS.values())
for symbol in top_20_pe_stocks[MARKET_COLS["symbol"]]:
  response = iex_cloud_api(f"stock/{symbol}/stats")
  stats_df.loc[symbol] = pd.Series([
    symbol,
    *response.values()
  ], index=STATS_COLS.values()) 
stats_df


,Symbol,Company Name,Market Cap,52 Week High,52 Week Low,52 Week High Split Adjust Only,52 Week Low Split Adjust Only,52 Week Change,Outstanding Shares,Float,...,Max Change %,5 Year Change %,2 Year Change %,1 Year Change %,Year-to-date Change %,6 Month Change %,3 Month Change %,1 Month Change %,30 Day Change %,5 Day Change %
TSLA,TSLA,Tesla Inc,1013082297401,1243.49,546.98,1243.49,546.98,0.401687,1004264852,0,...,210.13018,15.504908,6.344915,0.355668,-0.045421,0.128389,0.068736,0.095119,0.01489,0.024142
CRM,CRM,Salesforce Inc,175457700000,311.75,176.74,311.75,176.74,-0.233931,990000000,0,...,18.449108,1.115169,0.177139,-0.231306,-0.302601,-0.388461,-0.189361,-0.170737,-0.187801,-0.064305
BKNG,BKNG,Booking Holdings Inc,91954283230,2715.66,1796.45,2715.66,1796.45,-0.0459,41063485,0,...,50.348773,0.278143,0.66837,-0.050222,-0.066651,-0.071257,-0.045416,0.036056,0.016967,0.005401
DIS,DIS,Walt Disney Co (The),221498260417,189.22,121.28,189.22,121.28,-0.334318,1820633408,0,...,3.176479,0.110137,0.210066,-0.335591,-0.214539,-0.28995,-0.114427,-0.122349,-0.131682,-0.067525
TMUS,TMUS,T-Mobile US Inc,165031202923,150.2,101.51,150.2,101.51,0.002809,1249289954,0,...,7.101213,1.068497,0.534416,-0.000151,0.138989,0.130509,0.299941,0.048662,0.050998,-0.006468
KHC,KHC,Kraft Heinz Co,53895342248,44.48,32.44,44.95,32.78,0.120015,1224894142,0,...,-0.200531,-0.399064,0.688379,0.103584,0.238603,0.236063,0.201779,0.169591,0.15942,0.040435
NVDA,NVDA,NVIDIA Corp,587881446602,346.37,134.5,346.47,134.59,0.360051,2912755520,0,...,34.624393,7.035689,2.001788,0.314872,-0.313643,-0.110307,-0.136372,-0.245044,-0.239067,-0.050569
COST,COST,Costco Wholesale Corp,262273541365,612.27,362.69,612.27,365.29,0.605296,443224290,0,...,14.219454,2.877018,1.04661,0.593152,0.043926,0.243836,0.230533,0.065181,0.057774,0.002287
AMT,AMT,American Tower Corp.,121702027202,299.22,218.83,303.72,220,0.081582,456240027,0,...,4.459107,1.364285,0.146664,0.078837,-0.083159,-0.03416,0.095669,0.109816,0.118938,0.043868
LLY,LLY,Lilly(Eli) & Co,275809251161,314,175.91,314,178.58,0.571766,952347126,0,...,7.851215,2.929185,0.962361,0.539664,0.052853,0.209783,0.19649,-0.000104,0.016104,-0.040582


### Removing glamour stocks

In [20]:
both = pd.concat([top_20_pe_stocks, stats_df], axis=1)
both = both.loc[:, ~both.columns.duplicated()]
both[[MARKET_COLS[col] for col in ["symbol", "peRatio", "week52High", "volume"]]]

,Symbol,PE Ratio,52 Week High,Volume
TSLA,TSLA,207.21,1243.49,None
CRM,CRM,117.78,311.75,None
BKNG,BKNG,79.88,2715.66,None
DIS,DIS,71.96,189.22,None
TMUS,TMUS,53.78,150.2,None
KHC,KHC,53.65,44.48,None
NVDA,NVDA,51.77,346.37,None
COST,COST,47.03,612.27,None
AMT,AMT,46.74,299.22,None
LLY,LLY,46.48,314,None


In [7]:
final_dataframe.sort_values(COLUMNS[2], inplace = True)
final_dataframe = final_dataframe[final_dataframe[COLUMNS[2]] > 0][:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

NameError: name 'final_dataframe' is not defined

### Calculating the number of shares to buy

In [ ]:
portfolio_size = 2

In [ ]:
position_size = portfolio_size/len(final_dataframe)
for row in final_dataframe.index:
  final_dataframe.loc[row, COLUMNS[3]] = floor(position_size / final_dataframe.loc[row, COLUMNS[1]])
final_dataframe

,Ticker,Latest Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,F,15.48,3.48,0
1,COF,132.59,4.92,0
2,C,50.93,5.03,0
3,GS,321.64,5.41,0
4,DD,68.7,5.78,0
5,AIG,63.03,5.83,0
6,GM,40.13,5.99,0
7,WBA,44.81,6.18,0
8,T,19.54,7.08,0
9,DOW,65.49,7.77,0


## A more realistic strategy

With this method, we will use the following values to create our strategy

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise value divided by Earnings before interest, taxes, deprecation and amortization (EV/EBITDA)
* Enterprise value divided by Gross Profit (EV/GP)

### Single api call

In [ ]:
symbol = 'AAPL'
types = 'advanced-stats'
params= f'symbols={symbol}&types={types}&token={IEX_CLOUD_API_TOKEN}'
batch_api_url = f'{API_BASE}/{BATCH_API_EP}?{params}'
data = requests.get(batch_api_url).json()
adv_stats = data[symbol]['advanced-stats']

# Price-to-earnings ratio
pe_ratio = adv_stats['peRatio']

# Price-to-book ratio
pb_ratio = adv_stats['priceToBook']

# Price-to-sales ratio
ps_ratio = adv_stats['priceToSales']

#Enterprise value divided by Earnings before interest, taxes, deprecation and amortization (EV/EBITDA)
ev_to_ebitda = adv_stats['enterpriseValue'] / adv_stats['EBITDA']

#Enterprise value divided by Gross Profit (EV/GP)
ev_to_gp = adv_stats['enterpriseValue'] / adv_stats['grossProfit']

pp.pprint({
    'pe_ratio': pe_ratio, 
    'pb_ratio': pb_ratio, 
    'ps_ratio':ps_ratio, 
    'ev_to_ebitda': ev_to_ebitda, 
    'ev_to_gp': ev_to_gp
})

{ 'ev_to_ebitda': 24.341755451720832,
  'ev_to_gp': 18.222892317069064,
  'pb_ratio': 31.85,
  'pe_ratio': 31.939737223081785,
  'ps_ratio': 7.26}


In [ ]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [ ]:
types = 'quote,advanced-stats'

symbol_csv_chunks = chunks(stocks, 100)
symbol_csv_list = []

for i in range(0, len(symbol_chunks)):
    symbol_csv_list.append(','.join(symbol_chunks[i]))
    
for symbol_csv in symbol_csv_list:
    params= f'symbols={symbol_csv}&types={types}&token={IEX_CLOUD_API_TOKEN}'
    batch_api_url = f'{API_BASE}/{BATCH_API_EP}?{params}'
    data = requests.get(batch_api_url).json()

    for symbol in symbol_csv.split(','):
        quote = data[symbol]['quote']
        adv_stats = data[symbol]['advanced-stats']
        
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol, # Ticker
                    quote['latestPrice'], # Price
                    floor(position_size/quote['latestPrice']), # Number of Shares to Buy
                    adv_stats['peRatio'], # Price-to-earnings Ratio
                    'N/A', # PE Percentile
                    adv_stats['priceToBook'], # Price-to-Book Ratio
                    'N/A', # PB Percentile
                    adv_stats['priceToSales'], # Price-to-sales Ratio
                    'N/A', # PS Percentile
                    divide_check_for_none(adv_stats['enterpriseValue'], adv_stats['EBITDA']), # EV/EBITDA
                    'N/A', # EV/EBITDA Percentile
                    divide_check_for_none(adv_stats['enterpriseValue'], adv_stats['grossProfit']), # EV/GP
                    'N/A', # EV/GP Percentile
                    'N/A', # RV Score
                ],
                index = rv_columns
            ),
            ignore_index = True
        )
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,126.45,1581,48.678233,N/A,8.03,N/A,7.11,N/A,30.232171,N/A,13.242361,N/A,N/A
1,AAL,23.73,8428,-1.724898,N/A,-2.14,N/A,0.887,N/A,-4.467264,N/A,2.328124,N/A,N/A
2,AAP,193.57,1033,25.957606,N/A,3.56,N/A,1.26,N/A,12.380167,N/A,2.703519,N/A,N/A
3,AAPL,123.85,1614,32.64429,N/A,32.2,N/A,7.14,N/A,24.005723,N/A,18.023512,N/A,N/A
4,ABBV,107.37,1862,41.725936,N/A,14.5,N/A,4.24,N/A,13.74106,N/A,8.842531,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510,YUM,109.30,1829,36.676953,N/A,-4.18,N/A,5.85,N/A,24.395346,N/A,10.163196,N/A,N/A
1511,ZBH,162.79,1228,-252.45844,N/A,2.84,N/A,4.89,N/A,30.32078,N/A,8.560155,N/A,N/A
1512,ZBRA,479.58,417,51.70243,N/A,12.39,N/A,5.76,N/A,35.033003,N/A,13.142599,N/A,N/A
1513,ZION,57.51,3477,8.87294,N/A,1.29,N/A,2.63,N/A,6.814162,N/A,3.002063,N/A,N/A


In [ ]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
545,AON,236.260,846,None,N/A,None,N/A,None,N/A,N/A,N/A,N/A,N/A,N/A
547,APA,19.140,10449,None,N/A,None,N/A,None,N/A,N/A,N/A,N/A,N/A,N/A
576,BRK.B,267.640,747,None,N/A,None,N/A,None,N/A,N/A,N/A,N/A,N/A,N/A
623,CTL,11.000,18181,None,N/A,None,N/A,None,N/A,N/A,N/A,N/A,N/A,N/A
641,DISCK,36.330,5505,None,N/A,None,N/A,None,N/A,N/A,N/A,N/A,N/A,N/A
670,ETFC,50.370,3970,None,N/A,None,N/A,None,N/A,N/A,N/A,N/A,N/A,N/A
673,EVRG,59.270,3374,None,N/A,None,N/A,None,N/A,N/A,N/A,N/A,N/A,N/A
695,FOX,37.290,5363,None,N/A,None,N/A,None,N/A,N/A,N/A,N/A,N/A,N/A
697,FRC,174.230,1147,None,N/A,None,N/A,None,N/A,N/A,N/A,N/A,N/A,N/A
709,GOOG,2074.720,96,None,N/A,None,N/A,None,N/A,N/A,N/A,N/A,N/A,N/A


In [ ]:
percentile_columns = ['PE', 'PB', 'PS', 'EV/EBITDA', 'EV/GP']
price_columns = [
#     'Price-to-earnings Ratio',
#     'Price-to-Book Ratio',
#     'Price-to-sales Ratio',
    'EV/EBITDA',
#     'EV/GP'
]

# # To replace None values with 0
# # This needs to be done because stats.percentileofscore throws error when it encounters None
# # However, this correction alters the percentiles, a better solution is needed
# for row in rv_dataframe.index:
#     if rv_dataframe.loc[row, 'Price-to-earnings Ratio'] == None:
#         rv_dataframe.loc[row, 'Price-to-earnings Ratio'] = 0

for column in price_columns:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)
        
# for row in rv_dataframe.index:
#     for percentile_column in percentile_columns:
#         ratio_col = 'Price-to-earnings Ratio'
#         percentile_col = f'{percentile_column} Percentile'
#         rv_dataframe.loc[row, percentile_col] = score(rv_dataframe[ratio_col], rv_dataframe.loc[row, ratio_col])
# rv_dataframe
rv_dataframe

TypeError: unsupported operand type(s) for +: 'float' and 'str'